In [1]:
using Revise
using ConvexHullPricing
using LinearAlgebra
using Statistics
using DataFrames
using Plots, LaTeXStrings
using JLD2
using Plots.PlotMeasures
using ProgressBars, JuMP, Gurobi

function mon(array)
  result = Float64[]
  last = array[1]
  for elt in array
    new_last = minimum([elt, last])
    push!(result, new_last)
    last = new_last
  end
  return result
end

const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer

instances = []
for file in readdir("..//data//ca"; join=true)
  push!(instances, UT.load_data(file))
end

function get_last_elements(array)
  result = []
  for i in 1:5:100
    percentage = i / 100
    last_index = length(array) - Int(ceil(length(array) * percentage))
    last_elements = mean(array[last_index:end])
    push!(result, last_elements)
  end
  return (result)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01


get_last_elements (generic function with 1 method)

In [2]:
function getlast(array)
  percentage = 90 / 100
  last_index = Int(ceil(length(array) * percentage))
  last_elements = mean(array[last_index:end])
  return last_elements
end

getlast (generic function with 1 method)

In [3]:
methods = ["BundleLevelMethodCA", "BundleProximalLevelMethod-L-CA", "D-AdaptationCA", "DowGCA", "SUBG-EP-CA", "SubGCA"]
methods_name = ["Bundle Level Method", "Bundle Proximal Level Method", "D-Adaptation", "DowG", "Estimated Polyak stepsizes, subgradient method", "Subgradient method"]

6-element Vector{String}:
 "Bundle Level Method"
 "Bundle Proximal Level Method"
 "D-Adaptation"
 "DowG"
 "Estimated Polyak stepsizes, subgradient method"
 "Subgradient method"

In [4]:
for (j, meth) in enumerate(methods)
  all_values = []
  all_distances = []
  for idx=1:20
    if idx ∉ [9, 10, 11, 12]
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunCA$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      average_iterates = get_last_elements(ITBLM)
      values = []
      for avg in average_iterates
        push!(values, FS - UT.exact_oracle(instance, avg)[1])
      end
      push!(all_values, values)
      plot(
        title = "Belgian instance $(idx)",
        xlabel = "Percentage of last iterations considered " * " (%)",
        ylabel = L"\mathcal{L}_* - \mathcal{L}",
        xlims=(0,100),
        xticks=1:5:100,
        titlefontsize=10,
        legend=:topright,
        labelfontsize=10,
        size=(1000,500),
        bottom_margin = 15px,
        left_margin = 30px,
        fmt=:pdf
      )
      plot!(1:5:100, values, label="$(methods_name[j])")
      hline!([minimum(mon(FS .- (BLM)))], label = "Best iterate")
      push!(all_distances, minimum(mon(FS .- (BLM))))
      savefig("avg_to_10/$(meth)$(idx).pdf")
    end
  end
  plot(
    title = "Belgian dataset",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(0,100),
    xticks=1:5:100,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
  )
  plot!(1:5:100, mean(all_values), label="$(methods_name[j])")
  hline!([mean(all_distances)], label = "Best iterate")
  savefig("avg_to_10/$(meth).pdf")
end

In [56]:
all_values = []
all_distances = []
for idx=1:8
  instance = instances[idx]
  FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
  XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//FGMBE$(idx).jld2")
  average_iterates = get_last_elements(ITBLM)
  values = []
  for avg in average_iterates
    push!(values, FS - UT.exact_oracle(instance, avg)[1])
  end
  push!(all_values, values)
  plot(
    title = "Belgian instance $(idx)",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(0,10),
    xticks=1:1:10,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
)
  plot!(1:1:10, values, label="Fast Gradient method")
  hline!([minimum(mon(FS .- (BLM)))], label = "Best iterate")
  push!(all_distances, minimum(mon(FS .- (BLM))))
  savefig("avg_to_10/FGM-BE$(idx).pdf")
end

In [57]:
plot(
    title = "Belgian dataset",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(0,10),
    xticks=1:1:10,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
)
plot!(1:1:10, mean(all_values), label="Fast Gradient method")
hline!([mean(all_distances)], label = "Best iterate")
savefig("avg_to_10/FGM-BE.pdf")

"c:\\Users\\Sofiane\\Documents\\ConvexHullPricing.jl\\notebooks\\avg_to_10\\FGM-BE.pdf"

In [59]:
argmin(mean(all_values))

10